In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os

In [2]:
import sys
from pathlib import Path

# Add project root (parent of the optimization folder) to sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
THEATER_NAME = "AMC Boston Common 19"
NUM_SCREENS = 19
SCREENS = [f"Screen_{i}" for i in range(1, NUM_SCREENS+1)]

In [4]:
# Days and screens
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
OPENING_TIME = dt.time(11,0)
CLOSING_TIME=dt.time(23,0)
TICKET_PRICE = 11.31

TICKET_PRICES = [TICKET_PRICE]*len(DAYS)
today = dt.date.today()
open_dt = dt.datetime.combine(today, OPENING_TIME)
close_dt = dt.datetime.combine(today, CLOSING_TIME)
delta = close_dt - open_dt


# Operating hours per screen per day (e.g., 10 hours = 600 minutes)
OPERATING_MIN_PER_DAY = {
    day: delta.total_seconds()/60 for day in DAYS  # same for all days for now
}

# Theater capacities per screen
SCREEN_CAPACITIES = {s: 210 
for s in SCREENS[:19]}
SCREEN_CAPACITIES["Screen_19"] = 600

# Buffer time between shows (e.g., cleaning, trailers, turnover) in minutes
BUFFER_MIN = 15

# Max number of showings per movie per day (frequency constraint)
MAX_SHOWINGS_PER_MOVIE_PER_DAY = 5

# # Synthetic movie Data 
# movies_df = pd.DataFrame({
#     "movie_id": ["M1", "M2", "M3", "M4"],
#     "title": ["Blockbuster Action", "Indie Drama", "Family Animation", "Horror Thriller"],
#     "runtime_min": [130, 110, 95, 100],
#     "ticket_price": [15.0, 14.0, 13.0, 13.0]
# })

In [5]:
from tmdb_api_calling.utils import get_movies_out_now, get_movies_from_url

In [ ]:
tmdb_key = os.getenv("TMDB_API_KEY")
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_key}"
}

In [16]:
# CURRENT_MOVIES = get_movies_out_now(headers)

movies_df = pd.read_csv("../data/cleaned/final_merged_dataset_with_genres.csv")

In [19]:
current = movies_df[movies_df['release_date'] > '2025-10-01']

In [20]:
current

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,...,history,horror,music,mystery,romance,science_fiction,thriller,tv_movie,war,western
439,CMCSA,2025-11-10,Black Phone 2,Universal,551845,-0.59,-0.27,2943.0,188.0,70558650,...,0,1,0,0,0,0,1,0,0,0
1209,LGF.A,2025-11-10,Good Fortune,Lionsgate,70339,-0.53,-0.64,746.0,94.0,16188087,...,0,0,0,0,0,0,0,0,0,0
1656,Private,2025-11-06,Kiss of the Spider Woman,Roadside Att…,48,-0.83,-0.97,11.0,4.0,1623221,...,0,0,0,0,0,0,0,0,0,0
1692,Private,2025-11-06,Last Days,Vertical Ent…,664,-0.46,-0.84,74.0,9.0,219915,...,0,0,0,0,0,0,0,0,0,0
2322,Private,2025-10-19,Re-Election,Picturehouse,2215,-0.40,1.01,2.0,1108.0,19222,...,0,0,0,0,0,0,0,0,0,0
2344,PARA,2025-11-10,Regretting You,Paramount Pi…,811523,-0.50,-0.23,3196.0,254.0,38936925,...,0,0,0,0,1,0,0,0,0,0
2409,PARA,2025-11-10,Roofman,Paramount Pi…,52015,-0.49,-0.61,545.0,95.0,22376641,...,0,0,0,0,0,0,0,0,0,0
2537,Private,2025-11-09,Shelby Oaks,Neon,25000,-0.19,-0.87,200.0,125.0,4400918,...,0,1,0,1,0,0,1,0,0,0
2645,SONY,2025-11-10,Soul on Fire,Sony Pictures,9079,-0.69,-0.55,275.0,33.0,7349877,...,0,0,0,0,0,0,0,0,0,0
3300,Private,2025-11-06,The Mastermind,MUBI,15264,-0.08,-0.29,130.0,117.0,931510,...,0,0,0,0,0,0,0,0,0,0


In [9]:
movies_df = movies_df[(pd.to_datetime(movies_df.release_date)>pd.to_datetime("2025-10-20"))
&
(movies_df.original_language=="en")
]

In [10]:
movies_df.drop_duplicates(subset="original_title", inplace=True)

In [11]:
movies_df

,id,imdb_id,title,original_title,original_language,release_date,status,homepage,budget,revenue,...,origin_country,spoken_languages,genre_ids,genre_names,production_company_ids,production_company_names,production_company_countries,production_country_codes,production_country_names,belongs_to_collection
1,1363123,tt34276058,The Family Plan 2,The Family Plan 2,en,2025-11-11,Released,https://tv.apple.com/movie/umc.cmc.1wj0ab94s7t...,0,0,...,US,"Mandarin, English, French, Russian","28, 35","Action, Comedy","82819, 169668","Skydance Media, Municipal Pictures","US, US",US,United States of America,家庭计划（系列）
6,967941,tt19847976,Wicked: For Good,Wicked: For Good,en,2025-11-19,Released,https://www.wickedmovie.com,150000000,237743055,...,US,English,"14, 12, 10749","Fantasy, Adventure, Romance","33, 2527","Universal Pictures, Marc Platt Productions","US, US",US,United States of America,Wicked Collection
7,1084242,tt26443597,Zootopia 2,Zootopia 2,en,2025-11-26,Released,https://movies.disney.com/zootopia-2,180000000,0,...,US,English,"16, 10751, 35, 12, 9648","Animation, Family, Comedy, Adventure, Mystery",6125,Walt Disney Animation Studios,US,US,United States of America,Zootopia Collection
8,1242898,tt31227572,Predator: Badlands,Predator: Badlands,en,2025-11-05,Released,https://www.20thcenturystudios.com/movies/pred...,105000000,160221424,...,US,,"28, 878, 12","Action, Science Fiction, Adventure","127928, 840, 1302, 185309, 22213","20th Century Studios, Lawrence Gordon Producti...","US, US, US, US, US",US,United States of America,Predator Collection
10,1248226,tt31425731,Playdate,Playdate,en,2025-11-05,Released,https://www.amazon.com/gp/video/detail/B0FQ1MCFKJ,0,0,...,US,English,"28, 35, 10751","Action, Comedy, Family","110723, 28787, 221397, 215935, 55446, 376","Nickel City Pictures, Wide Awake Pictures, A H...","US, US, US, US, CA, US","US, CA","United States of America, Canada",None
12,425274,tt4712810,Now You See Me: Now You Don't,Now You See Me: Now You Don't,en,2025-11-12,Released,https://nowyouseeme.movie,90000000,146060630,...,US,English,"53, 80, 9648","Thriller, Crime, Mystery","1632, 281285, 221429","Lionsgate, Cohen Pictures, Media Capital Techn...","US, US, US",US,United States of America,Now You See Me Collection
14,1241983,tt29768334,Train Dreams,Train Dreams,en,2025-11-05,Released,https://www.netflix.com/title/82020378,10000000,0,...,US,"Mandarin, English","18, 37","Drama, Western","3540, 22146","Kamala Films, Black Bear Pictures","US, US",US,United States of America,None


# Optimization Formulation

1. Parameters

- $A_{i,d}$: expected attendance for movie i per day d

- $P_d$: average ticket price for movie on day d
- $D_i$: duration of movie i in minutes
- $B$: buffer time between shows
- $C_j$: capacity of screen j
- $H_{j,d}$: hours screen j operates on day d 
- $K$: maximum times a movie can be shown on a given day


2. Decision Variables:

- $s_{i,d}$: { 0, 1 }: whether movie i is scheduled on day d
- $x_{i, d, j}$: number of times we show movie i on screen j on a given day d 
- $r_{i, d}$: realized tickets served for movie i on day d, where $r_i,d=\min(j  x_i, d,j C_j,  A_i,d) $

3. Objective function: Maximize daily revenue:

$$max (\sum_ {i, d,j}x_{i, d, j}\times P_d r_{i,d}   ) $$


4. Constraints
- Total screen time: $\sum_{j} x_{i,d,j} (D_i+B) \leq H_{d,j} \forall d,j$
- Tickets served won’t exceed demand:  $r_i,d\leq \sum_{j} x_i, d,j C_j$
- Can’t sell more tickets than available: $r_i,d \leq A_i,d$  
- If not scheduled, no showings:  $\sum_{j} x_i, d,j \leq M*s_i,d$
- Each movie shown at most K times: $\sum_{j} x_i, d,j \leq K  ∀ i,d$
- Minimum number of genres each day: [TODO]

### Preprocessing

In [ ]:
runtimes = {}
demand = {}
for (i, row) in movies_df.iterrows():
    id = row['id']
    runtimes[id] = row['runtime']
    runtimes[id]= 
    # demand[id]


movie_ids = movies_df['id'].tolist()

In [51]:
from audioop import maxpp


def createModel(runtimes, demand, ticket_price, max_showings_per_day, operating_min_per_day, screen_capacity, movie_ids, days, screens, buffer_min, verbose=False):

    m = gp.Model("CinemaShowtimeScheduling")
    if not verbose:
        m.Params.OutputFlag = 0  # silence output by default
    
    x = m.addVars(
        movie_ids, days, screens,
        vtype=GRB.INTEGER,
        name="x"
    )
    
    # r[i,d] >= 0 continuous
    r = m.addVars(
        movie_ids, days,
        vtype=GRB.CONTINUOUS,
        lb=0.0,
        name="r"
    )
    
    # s[i,d] binary
    s = m.addVars(
        movie_ids, days,
        vtype=GRB.BINARY,
        name="s"
    )
    
    # Objective: maximize total revenue
    m.setObjective(
        gp.quicksum(ticket_price * r[i, d] for i in movie_ids for d in days),
        GRB.MAXIMIZE
    )
    
    # 1) Screen time constraint per screen & day
    #   sum_i x[i,d,j] * (runtime_i + buffer) <= operating_min_per_day[d]
    for d in days:
        for j in screens:
            m.addConstr(
                sum(x[i, d, j] * (runtimes[i] + buffer_min) for i in movie_ids)
                <= operating_min_per_day[d],
                name=f"Time_{d}_{j}"
            )
    
    # 2) Capacity: r[i,d] <= sum_j x[i,d,j] * capacity_j
    for i in movie_ids:
        for d in days:
            m.addConstr(
                r[i, d] <= gp.quicksum(x[i, d, j] * screen_capacity[j] for j in screens),
                name=f"Capacity_{i}_{d}"
            )
    
    # 3) Demand: r[i,d] <= A[i,d]
    for i in movie_ids:
        for d in days:
            demand_id = demand.get((i, d), 0)
            m.addConstr(
                r[i, d] <= demand_id,
                name=f"Demand_{i}_{d}"
            )
    
    # 4) Frequency + schedule linking
    for i in movie_ids:
        for d in days:
            # Frequency limit: total showings per day
            # Link to s: if s[i,d] = 0 => no showings
            m.addConstr(
                sum(x[i, d, j] for j in screens)
                <= max_showings_per_day * s[i, d],
                name=f"FreqLimit_{i}_{d}"
            )

    
         
    return m
    

In [52]:
model = createModel(runtimes, demand, TICKET_PRICE, MAX_SHOWINGS_PER_MOVIE_PER_DAY, OPERATING_MIN_PER_DAY,
SCREEN_CAPACITIES, movie_ids, DAYS, SCREENS, BUFFER_MIN, True)

In [53]:
model.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 280 rows, 1029 columns and 2940 nonzeros (Max)
Model fingerprint: 0x00862d22
Model has 49 linear objective coefficients
Variable types: 49 continuous, 980 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+02, 7e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 280 rows and 1029 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.00

In [62]:
def optimize_showtimes(model, verbose = False):

    model.optimize()

    x = model.getAttr('x')

    status_code = model.Status
    if status_code == GRB.OPTIMAL:
        status = "Optimal"
    elif status_code == GRB.INFEASIBLE:
        status = "Infeasible"
    elif status_code == GRB.UNBOUNDED:
        status = "Unbounded"
    else:
        status = f"Status_{status_code}"
    
    total_revenue = model.objVal if status_code == GRB.OPTIMAL else None
    
    if verbose:
        print("Status:", status)
        print("Total revenue:", total_revenue)
    
    schedule_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in DAYS:
                for j in SCREENS:
                    val = x[i, d, j].X
                    if val > 1e-6:
                        schedule_rows.append({
                            "movie_id": i,
                            "day": d,
                            "screen": j,
                            "showings": int(round(val))
                        })
    
    schedule_df = pd.DataFrame(schedule_rows)
    
    realized_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in DAYS:
                realized_rows.append({
                    "movie_id": i,
                    "day": d,
                    "realized_tickets": r[i, d].X,
                    "scheduled_flag": int(round(s[i, d].X))
                })
    realized_df = pd.DataFrame(realized_rows)
    
    return {
        "status": status,
        "total_revenue": total_revenue,
        "schedule_df": schedule_df,
        "realized_df": realized_df
    }
    

In [63]:
results = optimize_showtimes(model)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 280 rows, 1029 columns and 2940 nonzeros (Max)
Model fingerprint: 0x00862d22
Model has 49 linear objective coefficients
Variable types: 49 continuous, 980 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+02, 7e+02]


Presolve removed 280 rows and 1029 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: -0 -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


TypeError: list indices must be integers or slices, not tuple